In [1]:
import math
import pandas as pd
import datetime
from datetime import date, timedelta
from sklearn.feature_extraction.text import TfidfTransformer, TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline
import numpy as np
from nltk import sent_tokenize, word_tokenize
from nltk.tokenize import RegexpTokenizer
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /Users/mdyuldina/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

# Create dataset

In [2]:
df = pd.read_json('data/channel_messages.json')

In [3]:
df = df[df['message'].notna()]
df['user_id'] = df['from_id'].apply(lambda x: x['user_id'])
df['forwarded'] = df['fwd_from'].notna()
df['forwards'] = df['forwards'].apply(lambda x: x is not None and x > 0)
df['replies'] = df['replies'].apply(lambda x: x['replies'] if x else 0)
df['pinned'] = df['pinned'].apply(lambda x: x == 1.0)

In [4]:
df = df.sort_values(by='id')

In [5]:
columns = ['date','message','mentioned','media_unread','post','pinned', 'from_id', 'fwd_from', \
           'via_bot_id', 'reply_to', 'media', 'reply_markup', 'replies']
important_columns = ['date', 'message', 'pinned', 'user_id', 'forwarded', 'forwards']

In [6]:
df = df[important_columns]

In [7]:
all_messages = df['message'].to_list()
len(all_messages)

3249

In [8]:
users = pd.read_json('data/channel_users.json')

In [9]:
df = pd.merge(df, users[['id', 'user', 'is_bot']], how='left', left_on="user_id", right_on='id', suffixes=['_x', '_y'])
del df['id']

In [10]:
df['is_bot'] = df['is_bot'].apply(lambda x: x == True)
df['is_admin'] = df['user'].apply(lambda x: x in ['pavel_kikin', 'Irinka_Bekker', 'Chihvost']) 

# TF-IDF using sklearn

## Make short summary for every day

In [11]:
pipe = Pipeline([('count', CountVectorizer(stop_words=['привет', 'спасибо'])),
                 ('tfid', TfidfTransformer())]).fit(all_messages)

In [12]:
min_date = df['date'].min().date()

In [13]:
max_date = df['date'].max().date()

In [14]:
def boost_score(row):
    score = row['score']
    if row['pinned']:
        score = score * 3
    if row['is_admin']:
        score = score * 2
    return score

In [15]:
delta = timedelta(days=1)
cur_date = min_date
while cur_date <= max_date:
    print('***** ' + cur_date.strftime("%Y-%m-%d") + ' *****')
    
    sub_df = df[df['date'].dt.date == cur_date].copy()
    if len(sub_df) == 0:
        cur_date += delta
        continue
    sub_df = sub_df[~sub_df['is_bot']]
    sub_df = sub_df[['message', 'pinned', 'user', 'is_admin']]
    
    dataset = []
    for row in sub_df.iterrows():
        message = row[1][0]
        pinned = row[1][1]
        user = row[1][2]
        is_admin = row[1][3]
        for line in sent_tokenize(message):
            dataset.append([line, pinned, user, is_admin])
    dataset = pd.DataFrame(dataset, columns=['sentence', 'pinned', 'user', 'is_admin'])
    
    dataset = dataset.drop_duplicates(subset='sentence').copy()
    dataset['user'] = dataset['user'].fillna(value='unknown')
    sentences = dataset['sentence'].to_list()
    tdidf = pipe.transform(sentences)
    scores = np.sum(tdidf.toarray(), axis=1)
    dataset['score'] = scores
    dataset['score_updated'] = dataset[['score', 'pinned', 'is_admin']].apply(boost_score, axis=1)
    threshold = dataset['score_updated'].quantile(q=0.95)
    
    summary = ''
    prev_user = None
    for row in dataset[dataset['score_updated'] >= threshold].iterrows():
        user = row[1][2]
        if user is None:
            user = 'unknown'
        if not prev_user or prev_user != user:
            prev_user = row[1][2]
            summary = summary + '\n' + '@' + str(user) + '\n'
        
        summary = summary + row[1][0] + '\n'
    print(summary)

    cur_date += delta

***** 2022-02-28 *****

@Chihvost
Зовите меня в тех случаях, когда возникают вопросы о платформе в целом, сообществах, экспертах, проблемах с регистрацией и прочих технических вопросах @Chihvost

✨ Еще в чате есть главный организатор курса Павел Кикин @pavel_kikin, Алексей Колесников @alekseyntk и Ирина Беккер.
Там у нас есть сообщество Машинное обучение и Нейронные сети, где настоящие эксперты отвечают на злободневные вопросы про машинное обучение и смежные области, а также мы еженедельно обновляем дайджесты с новостями с It мероприятий, конкурсов и выкладываем вакансии.

***** 2022-03-01 *****
***** 2022-03-02 *****
***** 2022-03-03 *****
***** 2022-03-04 *****
***** 2022-03-05 *****
***** 2022-03-06 *****
***** 2022-03-07 *****
***** 2022-03-08 *****
***** 2022-03-09 *****

@Chihvost
🎉
Если есть желание, то можно рассказать немного о себе с тегом #знакомство 🤗

***** 2022-03-10 *****

@multi_it_mind
Всем привет!

***** 2022-03-11 *****
***** 2022-03-12 *****
***** 2022-03-13 *****
*


@DENisVali
@marcenavuc короч, как я услышал твой посыл, Илоном Маском, но в России ты становиться не хочешь, если ты создашь глобальный проект, адрес проекта будет не Россия 🇷🇺 а США, Калифорния 🇺🇸?

@pavel_kikin
Промежуточных отчетов не нужно, только общая защита в финале курса
Пойдёт и гитхаб, только придется самостоятельно разбираться в его CI/CD.
По курсу буду всё показывать в гитлабе.
Можно, просто назвав remote репозитории по разному.
Можно через конфиг настроить так, чтобы одной командой делать push

***** 2022-04-11 *****

@pavel_kikin
Сегодня рассмотрим Codestyle в Python, стандарт pep8 и автоформатеры, настройку линтеров в CI Gitlab.
Присоединяйтесь 😉 : 
18:30 МСК
youtube
zoom

***** 2022-04-12 *****

@Garvild
результат будет 1 и тот же(если смотреть на состояния репы после команд, а не на твою позицию в ней), просто команда git checkout -b <u_branch> = git branch <u_branch> &  git checkout <u_branch>, спрашивается как изменяется данные в репе в связи с этими командами

@Den

# Experimantal. Do not use.

In [16]:
sub_df = df[(df['date'] >= '2022-04-01') & (df['date'] < '2022-04-02')].copy()
sub_df = sub_df[~sub_df['is_bot']]

In [17]:
for message in sub_df['message'].tolist():
    print(message)

Доброе утро. @pavel_kikin, можно две темы для доклада совместить, рассказать про tensortt, onnx и квантизацию заодно?
Доброе утро. Готов. Направил желание участвовать в личку. Посмотрите, пожалуйста.
А не хочешь объедениться для этой темы? Я бы ещё triton добавил бы сюда, было бы в тему
Тритон это скорее к tf serving
Кстати, а кто уже пробовал serving dl моделей? Какое у вас получалось среднее время на ответ?
Так от модели, сервера, железа всё зависит
Знающие люди, подскажите начинающему:
1) Насколько понял, для прохождения курса понадобится какой-либо существующий DS проект. Если своего проекта нет, чем можно воспользоваться для обучения.
2) Выполнение заданий предполагается локально или в облаке? Достаточно ли будет для локального выполнения ноута с GeForce MX 130? Если облако, то посоветуйте что именно и где (желательно подешевле).
Заранее спасибо и не судите строго - совсем новичок)
>  Если своего проекта нет, чем можно воспользоваться для обучения.
Возможно, что-то интересное найд

In [18]:
sub_df = sub_df[['message', 'pinned', 'user', 'is_admin']]
sub_df

,message,pinned,user,is_admin
684,"Доброе утро. @pavel_kikin, можно две темы для ...",False,starminalush,False
685,Доброе утро. Готов. Направил желание участвова...,False,dizel0110,False
686,А не хочешь объедениться для этой темы? Я бы е...,False,ivanglebov,False
687,Тритон это скорее к tf serving,False,starminalush,False
688,"Кстати, а кто уже пробовал serving dl моделей?...",False,marcenavuc,False
...,...,...,...,...
889,У нас явно сложилась взаимная симпатия,False,YuryFilimonov,False
890,А я не нуждаюсь в ней пока что. Но если нужно ...,False,redpf,False
891,Я так и подумала,False,YuryFilimonov,False
892,"Ой, подумал. Тоже перепутал.",False,YuryFilimonov,False


In [19]:
dataset = []
for row in sub_df.iterrows():
    message = row[1][0]
    pinned = row[1][1]
    user = row[1][2]
    is_admin = row[1][3]
    for line in sent_tokenize(message):
        dataset.append([line, pinned, user, is_admin])
dataset = pd.DataFrame(dataset, columns=['sentence', 'pinned', 'user', 'is_admin'])
dataset

,sentence,pinned,user,is_admin
0,Доброе утро.,False,starminalush,False
1,"@pavel_kikin, можно две темы для доклада совме...",False,starminalush,False
2,Доброе утро.,False,dizel0110,False
3,Готов.,False,dizel0110,False
4,Направил желание участвовать в личку.,False,dizel0110,False
...,...,...,...,...
393,Но если нужно будет я добавлюсь обязательно,False,redpf,False
394,Я так и подумала,False,YuryFilimonov,False
395,"Ой, подумал.",False,YuryFilimonov,False
396,Тоже перепутал.,False,YuryFilimonov,False


In [20]:
dataset = dataset.drop_duplicates(subset='sentence').copy()
dataset

,sentence,pinned,user,is_admin
0,Доброе утро.,False,starminalush,False
1,"@pavel_kikin, можно две темы для доклада совме...",False,starminalush,False
3,Готов.,False,dizel0110,False
4,Направил желание участвовать в личку.,False,dizel0110,False
5,"Посмотрите, пожалуйста.",False,dizel0110,False
...,...,...,...,...
393,Но если нужно будет я добавлюсь обязательно,False,redpf,False
394,Я так и подумала,False,YuryFilimonov,False
395,"Ой, подумал.",False,YuryFilimonov,False
396,Тоже перепутал.,False,YuryFilimonov,False


In [21]:
dataset['user'] = dataset['user'].fillna(value='unknown')

In [22]:
sentences = dataset['sentence'].to_list()

In [23]:
tokenizer = RegexpTokenizer('https?://[/\.\?=\S]+|\w+')

def _create_frequency_matrix(sentences):
    frequency_matrix = {}
    stopWords = set(['привет', 'спасибо'])

    for sent in sentences:
        freq_table = {}
        words = tokenizer.tokenize(sent)
        for word in words:
            word = word.lower()
            if word in stopWords:
                continue

            if word in freq_table:
                freq_table[word] += 1
            else:
                freq_table[word] = 1

        frequency_matrix[sent[:15]] = freq_table

    return frequency_matrix

In [24]:
freq_matrix = _create_frequency_matrix(sentences)
freq_matrix

{'Доброе утро.': {'доброе': 1, 'утро': 1},
 '@pavel_kikin, м': {'pavel_kikin': 1,
  'можно': 1,
  'две': 1,
  'темы': 1,
  'для': 1,
  'доклада': 1,
  'совместить': 1,
  'рассказать': 1,
  'про': 1,
  'tensortt': 1,
  'onnx': 1,
  'и': 1,
  'квантизацию': 1,
  'заодно': 1},
 'Готов.': {'готов': 1},
 'Направил желани': {'направил': 1,
  'желание': 1,
  'участвовать': 1,
  'в': 1,
  'личку': 1},
 'Посмотрите, пож': {'посмотрите': 1, 'пожалуйста': 1, 'личку': 1},
 'А не хочешь объ': {'а': 1,
  'не': 1,
  'хочешь': 1,
  'объедениться': 1,
  'для': 1,
  'этой': 1,
  'темы': 1},
 'Я бы ещё triton': {'я': 1,
  'бы': 3,
  'ещё': 1,
  'triton': 1,
  'добавил': 1,
  'сюда': 1,
  'было': 1,
  'в': 1,
  'тему': 1},
 'Тритон это скор': {'тритон': 1,
  'это': 1,
  'скорее': 1,
  'к': 1,
  'tf': 1,
  'serving': 1},
 'Кстати, а кто у': {'кстати': 1,
  'а': 1,
  'кто': 1,
  'уже': 1,
  'пробовал': 1,
  'serving': 1,
  'dl': 1,
  'моделей': 1},
 'Какое у вас пол': {'какое': 1,
  'у': 1,
  'вас': 1,
  'п

In [25]:
def _create_tf_matrix(freq_matrix):
    tf_matrix = {}

    for sent, f_table in freq_matrix.items():
        tf_table = {}

        count_words_in_sentence = len(f_table)
        for word, count in f_table.items():
            tf_table[word] = count / count_words_in_sentence

        tf_matrix[sent] = tf_table

    return tf_matrix

In [26]:
tf_matrix = _create_tf_matrix(freq_matrix)
tf_matrix

{'Доброе утро.': {'доброе': 0.5, 'утро': 0.5},
 '@pavel_kikin, м': {'pavel_kikin': 0.07142857142857142,
  'можно': 0.07142857142857142,
  'две': 0.07142857142857142,
  'темы': 0.07142857142857142,
  'для': 0.07142857142857142,
  'доклада': 0.07142857142857142,
  'совместить': 0.07142857142857142,
  'рассказать': 0.07142857142857142,
  'про': 0.07142857142857142,
  'tensortt': 0.07142857142857142,
  'onnx': 0.07142857142857142,
  'и': 0.07142857142857142,
  'квантизацию': 0.07142857142857142,
  'заодно': 0.07142857142857142},
 'Готов.': {'готов': 1.0},
 'Направил желани': {'направил': 0.2,
  'желание': 0.2,
  'участвовать': 0.2,
  'в': 0.2,
  'личку': 0.2},
 'Посмотрите, пож': {'посмотрите': 0.3333333333333333,
  'пожалуйста': 0.3333333333333333,
  'личку': 0.3333333333333333},
 'А не хочешь объ': {'а': 0.14285714285714285,
  'не': 0.14285714285714285,
  'хочешь': 0.14285714285714285,
  'объедениться': 0.14285714285714285,
  'для': 0.14285714285714285,
  'этой': 0.14285714285714285,
  '

In [27]:
def _create_documents_per_words(freq_matrix):
    word_per_doc_table = {}

    for sent, f_table in freq_matrix.items():
        for word, count in f_table.items():
            if word in word_per_doc_table:
                word_per_doc_table[word] += 1
            else:
                word_per_doc_table[word] = 1

    return word_per_doc_table

In [28]:
count_doc_per_words = _create_documents_per_words(freq_matrix)
count_doc_per_words

{'доброе': 1,
 'утро': 1,
 'pavel_kikin': 3,
 'можно': 22,
 'две': 1,
 'темы': 2,
 'для': 29,
 'доклада': 1,
 'совместить': 1,
 'рассказать': 3,
 'про': 13,
 'tensortt': 1,
 'onnx': 1,
 'и': 84,
 'квантизацию': 1,
 'заодно': 1,
 'готов': 5,
 'направил': 1,
 'желание': 2,
 'участвовать': 1,
 'в': 83,
 'личку': 7,
 'посмотрите': 2,
 'пожалуйста': 1,
 'а': 37,
 'не': 77,
 'хочешь': 1,
 'объедениться': 1,
 'этой': 1,
 'я': 51,
 'бы': 14,
 'ещё': 11,
 'triton': 1,
 'добавил': 1,
 'сюда': 2,
 'было': 10,
 'тему': 6,
 'тритон': 1,
 'это': 36,
 'скорее': 1,
 'к': 14,
 'tf': 2,
 'serving': 2,
 'кстати': 3,
 'кто': 12,
 'уже': 17,
 'пробовал': 1,
 'dl': 1,
 'моделей': 1,
 'какое': 4,
 'у': 22,
 'вас': 6,
 'получалось': 1,
 'среднее': 1,
 'время': 6,
 'на': 43,
 'ответ': 1,
 'так': 17,
 'от': 13,
 'модели': 1,
 'сервера': 1,
 'железа': 1,
 'всё': 5,
 'зависит': 1,
 'знающие': 1,
 'люди': 4,
 'подскажите': 2,
 'начинающему': 1,
 '1': 6,
 'насколько': 5,
 'понял': 3,
 'прохождения': 1,
 'курса': 4,

In [29]:
def _create_idf_matrix(freq_matrix, count_doc_per_words, total_documents):
    idf_matrix = {}

    for sent, f_table in freq_matrix.items():
        idf_table = {}

        for word in f_table.keys():
            idf_table[word] = math.log10(total_documents / float(count_doc_per_words[word]))

        idf_matrix[sent] = idf_table

    return idf_matrix

In [30]:
idf_matrix = _create_idf_matrix(freq_matrix, count_doc_per_words, len(sentences))
idf_matrix

{'Доброе утро.': {'доброе': 2.578639209968072, 'утро': 2.578639209968072},
 '@pavel_kikin, м': {'pavel_kikin': 2.10151795524841,
  'можно': 1.236216529145866,
  'две': 2.578639209968072,
  'темы': 2.2776092143040914,
  'для': 1.1162412120691163,
  'доклада': 2.578639209968072,
  'совместить': 2.578639209968072,
  'рассказать': 2.10151795524841,
  'про': 1.4646958576612357,
  'tensortt': 2.578639209968072,
  'onnx': 2.578639209968072,
  'и': 0.6543599239061907,
  'квантизацию': 2.578639209968072,
  'заодно': 2.578639209968072},
 'Готов.': {'готов': 1.8796692056320534},
 'Направил желани': {'направил': 2.578639209968072,
  'желание': 2.2776092143040914,
  'участвовать': 2.578639209968072,
  'в': 0.6595611175919984,
  'личку': 1.7335411699538155},
 'Посмотрите, пож': {'посмотрите': 2.2776092143040914,
  'пожалуйста': 2.578639209968072,
  'личку': 1.7335411699538155},
 'А не хочешь объ': {'а': 1.0104374859010774,
  'не': 0.6921484847955904,
  'хочешь': 2.578639209968072,
  'объедениться': 

In [31]:
def _create_tf_idf_matrix(tf_matrix, idf_matrix):
    tf_idf_matrix = {}

    for (sent1, f_table1), (sent2, f_table2) in zip(tf_matrix.items(), idf_matrix.items()):

        tf_idf_table = {}

        for (word1, value1), (word2, value2) in zip(f_table1.items(),
                                                    f_table2.items()):  # here, keys are the same in both the table
            tf_idf_table[word1] = float(value1 * value2)

        tf_idf_matrix[sent1] = tf_idf_table

    return tf_idf_matrix

In [32]:
tf_idf_matrix = _create_tf_idf_matrix(tf_matrix, idf_matrix)
tf_idf_matrix

{'Доброе утро.': {'доброе': 1.289319604984036, 'утро': 1.289319604984036},
 '@pavel_kikin, м': {'pavel_kikin': 0.15010842537488642,
  'можно': 0.08830118065327613,
  'две': 0.18418851499771943,
  'темы': 0.16268637245029224,
  'для': 0.07973151514779402,
  'доклада': 0.18418851499771943,
  'совместить': 0.18418851499771943,
  'рассказать': 0.15010842537488642,
  'про': 0.10462113269008826,
  'tensortt': 0.18418851499771943,
  'onnx': 0.18418851499771943,
  'и': 0.0467399945647279,
  'квантизацию': 0.18418851499771943,
  'заодно': 0.18418851499771943},
 'Готов.': {'готов': 1.8796692056320534},
 'Направил желани': {'направил': 0.5157278419936144,
  'желание': 0.4555218428608183,
  'участвовать': 0.5157278419936144,
  'в': 0.13191222351839968,
  'личку': 0.3467082339907631},
 'Посмотрите, пож': {'посмотрите': 0.7592030714346971,
  'пожалуйста': 0.8595464033226907,
  'личку': 0.5778470566512718},
 'А не хочешь объ': {'а': 0.14434821227158248,
  'не': 0.09887835497079862,
  'хочешь': 0.3683

In [33]:
def _score_sentences(tf_idf_matrix) -> dict:
    """
    score a sentence by its word's TF
    Basic algorithm: adding the TF frequency of every non-stop word in a sentence divided by total no of words in a sentence.
    :rtype: dict
    """

    sentenceValue = {}

    for sent, f_table in tf_idf_matrix.items():
        total_score_per_sentence = 0

        count_words_in_sentence = len(f_table)
        for word, score in f_table.items():
            total_score_per_sentence += score

        #sentenceValue[sent] = total_score_per_sentence / count_words_in_sentence if count_words_in_sentence > 0 else 0
        sentenceValue[sent] = total_score_per_sentence 
    return sentenceValue


In [34]:
sentenceValue = _score_sentences(tf_idf_matrix)
sentenceValue

{'Доброе утро.': 2.578639209968072,
 '@pavel_kikin, м': 2.0716166512399874,
 'Готов.': 1.8796692056320534,
 'Направил желани': 1.9655979843572098,
 'Посмотрите, пож': 2.1965965314086597,
 'А не хочешь объ': 1.8331934324248702,
 'Я бы ещё triton': 2.0199361114371355,
 'Тритон это скор': 2.027890788672491,
 'Кстати, а кто у': 1.9966413172335051,
 'Какое у вас пол': 1.936857506405942,
 'Так от модели, ': 2.143873170250768,
 'Знающие люди, п': 2.1507598492355684,
 'Если своего про': 1.7407864354510012,
 '2) Выполнение з': 2.0793110782469966,
 'Достаточно ли б': 2.079749591987221,
 'Если облако, то': 1.7946962122755474,
 'Заранее спасибо': 1.9482349238877399,
 '>  Если своего ': 1.7407864354510012,
 'Возможно, что-т': 1.8003179812073018,
 'На кегле щас ах': 2.1017620986897914,
 'скинь ссылочку)': 2.578639209968072,
 'По compute capa': 1.96344650342154,
 'Памяти не так м': 1.7174716468878128,
 'Но можно же и н': 1.7014023466585422,
 'Какой-нибудь fr': 2.0548843161078785,
 'или взять колаб': 

In [35]:
def _find_average_score(sentenceValue) -> int:
    """
    Find the average score from the sentence value dictionary
    :rtype: int
    """
    sumValues = 0
    for entry in sentenceValue:
        sumValues += sentenceValue[entry]

    # Average value of a sentence from original summary_text
    average = (sumValues / len(sentenceValue))

    return average

In [36]:
avarage = _find_average_score(sentenceValue)
avarage

1.9058468318871509

In [37]:
def _generate_summary(sentences, sentenceValue, threshold):
    sentence_count = 0
    summary = ''

    for sentence in sentences:
        if sentence[:15] in sentenceValue and sentenceValue[sentence[:15]] >= (threshold):
            
            for word, value in tf_idf_matrix[sentence[:15]].items():
                if value >= 0.07:
                     summary += " " + word
                        
            
            summary += ". " 
            
            sentence_count += 1

    return summary

In [38]:
summary = _generate_summary(sentences, sentenceValue, 1.05 * avarage)
print(summary)

 доброе утро.  pavel_kikin можно две темы для доклада совместить рассказать про tensortt onnx квантизацию заодно.  посмотрите пожалуйста личку.  я бы ещё triton добавил сюда было в тему.  тритон это скорее к tf serving.  так от модели сервера железа всё зависит.  знающие люди подскажите начинающему 1 насколько понял прохождения курса понадобится какой либо существующий ds проект.  2 выполнение заданий предполагается локально или в облаке.  достаточно ли будет для локального выполнения ноута с geforce mx 130.  на кегле щас ахриненный сорев с птичками где со звуком работать нужно.  скинь ссылочку.  какой нибудь fraud detection и credit scoring табличными данными будет требовать меньше времени ресурсов обучения.  или взять колаб.  живу в сургуте могу присоединиться.  сургут норм.  в основном физические было вроде про концентрацию углекислого газа воздухе еще валялся датасет турбины чем суть помню.  замутим бейзлайн и еще какую нибудь модельку.  край можно нлп.  временные ряды мне тоже инт

In [39]:
total_documents = len(sentences)
freq_matrix = _create_frequency_matrix(sentences)
tf_matrix = _create_tf_matrix(freq_matrix)
count_doc_per_words = _create_documents_per_words(freq_matrix)
idf_matrix = _create_idf_matrix(freq_matrix, count_doc_per_words, total_documents)
tf_idf_matrix = _create_tf_idf_matrix(tf_matrix, idf_matrix)
sentence_scores = _score_sentences(tf_idf_matrix)
threshold = _find_average_score(sentence_scores)
summary = _generate_summary(sentences, sentence_scores, 0.8 * threshold)

In [40]:
summary

' доброе утро.  pavel_kikin можно две темы для доклада совместить рассказать про tensortt onnx квантизацию заодно.  готов.  направил желание участвовать в личку.  посмотрите пожалуйста личку.  а не хочешь объедениться для этой темы.  я бы ещё triton добавил сюда было в тему.  тритон это скорее к tf serving.  кстати а кто уже пробовал serving dl моделей.  какое у вас получалось среднее время на ответ.  так от модели сервера железа всё зависит.  знающие люди подскажите начинающему 1 насколько понял прохождения курса понадобится какой либо существующий ds проект.  если своего проекта нет чем можно воспользоваться для обучения.  2 выполнение заданий предполагается локально или в облаке.  достаточно ли будет для локального выполнения ноута с geforce mx 130.  если облако то посоветуйте что именно где желательно подешевле.  заранее и не судите строго совсем новичок.  если своего проекта нет чем можно воспользоваться для обучения.  возможно что то интересное найдется на каггле.  на кегле щас а